# Dataset split

The goal of this notebook is to create **train** and **validation** sets and check their consistency.

**NOTE**: To display this notebook properly [Nbextensions](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html) should be present and `Python Markdown` activated.

In [1]:
from footvid.utils.env import check_repository_path


REPOSITORY_DIR = check_repository_path()
RAW_DATA_DIR = REPOSITORY_DIR.joinpath("data", "raw")
PROCESSED_DATA_DIR = REPOSITORY_DIR.joinpath("data", "processed")

N_POS = 1643
N_NEG = 1335
N_TOTAL = N_POS + N_NEG

As we know from the [previous notebook](https://github.com/mrtovsky/footvid/blob/master/notebooks/00-images-integrity.ipynb) whole dataset contains {{ N_POS }} and {{ N_NEG }} negative observations. By manual investigation it has been discovered that the intersection of sets of matches contained in the `FrameFilter-set1_4k`, `FrameFilter-set2_fifawc2018` and the `FrameFilter-set2_fifawc2018` is not empty. Those folders will be merged together and among the images inside two sets will be created but in such a way that the frames from the matches included in the training set are not included in the validation set as well, what can potentially lead to a data leakage. `match_hash` field comes with help.

In [2]:
from collections import Counter, defaultdict

from IPython.display import display


# Store paths to image files.
pos_image_files = []
neg_image_files = []

for data_folder in sorted(RAW_DATA_DIR.glob("*")):
    if data_folder.is_dir():
        for label_folder in data_folder.glob("*"):
            if label_folder.is_dir():
                if label_folder.name == "neg":
                    neg_image_files.extend(list(label_folder.glob("*")))
                elif label_folder.name == "pos":
                    pos_image_files.extend(list(label_folder.glob("*")))

match_hash_counter = defaultdict(Counter)
for file in pos_image_files:
    match_hash_counter[file.stem.split("-")[1]]["n_pos"] += 1
for file in neg_image_files:
    match_hash_counter[file.stem.split("-")[1]]["n_neg"] += 1

print("`match_hash` counted occurrences by classes:\n")
display(match_hash_counter)

`match_hash` counted occurrences by classes:



defaultdict(collections.Counter,
            {'0989f5f192a285762fa1bd04': Counter({'n_pos': 39, 'n_neg': 45}),
             'd065ce1ce94a49804498e1f0': Counter({'n_pos': 5, 'n_neg': 17}),
             'bf8ccbeb6c3636c6b857af94': Counter({'n_pos': 46, 'n_neg': 42}),
             'e454a13b26a10fe08ef05181': Counter({'n_pos': 16, 'n_neg': 24}),
             'ee2c6f24bd9845218577df88': Counter({'n_pos': 34, 'n_neg': 24}),
             '7e9a7a63ca577c8d974622fd': Counter({'n_pos': 22, 'n_neg': 17}),
             '8feb9679c3d7e2f8f95fc179': Counter({'n_pos': 26, 'n_neg': 27}),
             '58cf64110dc8c1634cb69e94': Counter({'n_pos': 18, 'n_neg': 26}),
             '2a2f6aebee678fa1b8de5e77': Counter({'n_pos': 23, 'n_neg': 30}),
             '3e7265e32a67a480085ae3e9': Counter({'n_pos': 46, 'n_neg': 40}),
             'a09951a67b79d2a6a79c2110': Counter({'n_pos': 26, 'n_neg': 22}),
             'ff16070272e1ab1ce5615d62': Counter({'n_pos': 20, 'n_neg': 32}),
             'cda0284e800d207758

Now it's all about selecting the proper $train : valid$ proportion and dividing frames into train and validation set by searching the combination that is closer to the proportions specified. $7 : 3$ seems to be a reasonable split.

In [3]:
from collections import namedtuple


DatasetSize = namedtuple("DatasetSize", ["n_pos", "n_neg"])

pos_percentage = N_POS / (N_POS + N_NEG)
theoretical_train_size = DatasetSize(
    n_pos=int(N_TOTAL * pos_percentage * 0.7),
    n_neg=int(N_TOTAL * (1 - pos_percentage) * 0.7),
)
theoretical_valid_size = DatasetSize(
    n_pos=int(N_TOTAL * pos_percentage * 0.3),
    n_neg=int(N_TOTAL * (1 - pos_percentage) * 0.3),
)

print(
    "Theoretical splits are characterized by the numbers below:\n"
    f"* TRAIN: {theoretical_train_size.n_pos} positives, {theoretical_train_size.n_neg} negatives.\n"
    f"* VALID: {theoretical_valid_size.n_pos} positives, {theoretical_valid_size.n_neg} negatives."
)
keys = list(match_hash_counter.keys())

Theoretical splits are characterized by the numbers below:
* TRAIN: 1150 positives, 934 negatives.
* VALID: 492 positives, 400 negatives.


**NOTE:** Paragraph below contains LateX equation that may not render properly on GitHub.

Knowing the aforementioned numbers will let us optimize the division of frames across train and valid sets by, in an ideal world, considering all of the possible combinations of dividing matches by hashes and minimizing the following loss:

$$L := \sum_{\textrm{c, d } \in \textbf{ C } \times \textbf{ D}}{(actual_{c,d} - theoretical_{c, d})^2}$$
$$\textrm{where: }$$
$$\textbf{C} = \{\textrm{positive, negative}\} \textrm{,}$$
$$\textbf{D} = \{\textrm{train, valid}\}\textrm{.}$$

In general, we want to find the split that is closest to the calculated theoretical splits, by penalizing greater deviations from the theoretical values harder.

Due to the fact that the number of unique matches is {{ len(keys) }} it gives more than 33 million combinations - $2^{25}$. Thus, we will apply an algorithm that works with the $O(n^2)$ worst-case time complexity. First, we will sort hashes by a total size increasingly and then we will include one hash after another until the theoretical shape is exceeded. Then we will take a n-steps back and check if replacing last-n hashes with the current one will improve the **validation set** proportions - this is another simplification, we will only try to optimize the validation set proportions.

In [4]:
MatchHash = namedtuple("MatchHash", ["match_hash", "n_pos", "n_neg", "n_total"])

match_hashes = [
    MatchHash(
        match_hash=key,
        n_pos=value["n_pos"],
        n_neg=value["n_neg"],
        n_total=value["n_pos"] + value["n_neg"],
    )
    for key, value in match_hash_counter.items()
] 

match_hashes.sort(key=lambda tup: tup[-1])


def calculate_loss(
    match_hashes,
    theoretical_n_pos=theoretical_valid_size.n_pos,
    theoretical_n_neg=theoretical_valid_size.n_neg,
):
    n_pos_sum = sum([match_hash.n_pos for match_hash in match_hashes])
    n_neg_sum = sum([match_hash.n_neg for match_hash in match_hashes])
    
    return (theoretical_n_pos - n_pos_sum) ** 2 + (theoretical_n_neg - n_neg_sum) ** 2


best_valid_hashes = [match_hashes[0],]
loss = calculate_loss(best_valid_hashes)
for match_hash in match_hashes[1:]:
    best_hashes_n_pos_sum = sum([best_hash.n_pos for best_hash in best_valid_hashes])
    best_hashes_n_neg_sum = sum([best_hash.n_neg for best_hash in best_valid_hashes])
    if (
        best_hashes_n_pos_sum < theoretical_valid_size.n_pos
        and best_hashes_n_neg_sum < theoretical_valid_size.n_neg
    ):
        best_valid_hashes.append(match_hash)
        loss = calculate_loss(best_valid_hashes)
    else:
        challenger_best_loss = 9e9
        challenger_best_idx = None
        for idx in range(1, len(best_valid_hashes)):
            challenger_valid_hashes = best_valid_hashes[:-idx] + [match_hash,]
            challenger_loss = calculate_loss(challenger_valid_hashes)
            if challenger_loss < loss and challenger_loss < challenger_best_loss:
                challenger_best_loss = challenger_loss
                challenger_best_idx = idx
        if challenger_best_idx is not None:
            best_valid_hashes = best_valid_hashes[:-challenger_best_idx] + [match_hash,]
            loss = challenger_best_loss

This way we have extracted the most suitable combination of match hashes to construct the validation set.

In [5]:
print(
    "The validation set is characterized by the following numbers:\n"
    f"* positives: {sum([best_hash.n_pos for best_hash in best_valid_hashes])},\n"
    f"* negatives: {sum([best_hash.n_neg for best_hash in best_valid_hashes])},\n"
    f"* total: {sum([best_hash.n_total for best_hash in best_valid_hashes])}."
)

The validation set is characterized by the following numbers:
* positives: 469,
* negatives: 442,
* total: 911.


Those numbers are pretty close to the theoretical values, so we can assume that it is a fair division.

# Files segregation

When the match hashes that make up the validation set are known it is time to properly segregate raw files into **train** and **valid** folders. Those folders should be a subdirectory of the **processed** directory and the structure should look as follows:

In [6]:
!tree -d ../data/processed/

../data/processed/
├── train
│   ├── neg
│   └── pos
└── valid
    ├── neg
    └── pos

6 directories


In [7]:
import shutil


valid_match_hashes = set([valid_hash.match_hash for valid_hash in best_valid_hashes])

for file in pos_image_files:
    if file.stem.split("-")[1] in valid_match_hashes:
        shutil.copy(file, PROCESSED_DATA_DIR.joinpath("valid", "pos"))
    else:
        shutil.copy(file, PROCESSED_DATA_DIR.joinpath("train", "pos"))

for file in neg_image_files:
    if file.stem.split("-")[1] in valid_match_hashes:
        shutil.copy(file, PROCESSED_DATA_DIR.joinpath("valid", "neg"))
    else:
        shutil.copy(file, PROCESSED_DATA_DIR.joinpath("train", "neg"))